In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import torch
from torch.distributions import Normal, MultivariateNormal
from torch import matmul

## Univariate Normal Distribution

In [2]:
from torch.distributions import Normal

mu = torch.tensor([0.0], dtype=torch.float)
sigma = torch.tensor([5.0], dtype=torch.float)

uvn_dist = Normal(mu, sigma)

X = torch.tensor([0.0], dtype=torch.float)

# Log prob

def formula(X, mu, sigma):
    K = 1 / (math.sqrt(2 * math.pi) * sigma)
    E = math.exp( -1 * (X - mu) ** 2 * (1 / (2 * sigma ** 2)))
    return torch.log(K * E)

log_prob = uvn_dist.log_prob(X)

formula_log_prob = formula(X, mu, sigma)

assert torch.isclose(log_prob, formula_log_prob, atol=1e-4)

# Sampling

num_samples = 100000

samples = uvn_dist.sample([num_samples])

# Mean
    
sample_mean = samples.mean()

dist_mean = uvn_dist.mean

assert torch.isclose(sample_mean, dist_mean, atol=0.2)

# Variance

sample_var = uvn_dist.sample([num_samples]).var()

dist_var = uvn_dist.variance

assert torch.isclose(sample_var, dist_var, atol=0.2)

Here we allow the user to set different values for the mean and variance of a univariate normal distribution and visualise the resulting distribution. 
Specifically, notice that changing the mean does not change the shape of the distribution. It just varies where the distribution peaks. Changing the variance causes the distribution to either become more diffuse / peaked.

In [3]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

fig, ax = plt.subplots()

ax.set_title("Univariate normal distribution")
ax.set_ylabel("P(X)")
ax.set_xlabel("X")


@interact
def plot_univariate_normal(mu=(-20, 20, 0.5), sigma=(1, 30, 0.5)):
    x = np.linspace(mu - 3*sigma, mu + 3*sigma, 1000)
    [l.remove() for l in ax.lines]
    uvn_dist = Normal(mu, sigma)
    pdf = uvn_dist.log_prob(torch.from_numpy(x)).exp()
    ax.set_ylim(0, 1.5 * max(pdf))
    ax.plot(x, pdf)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=0.0, description='mu', max=20.0, min=-20.0, step=0.5), FloatSlider(val…

## Multivariate Normal Distribution

In [4]:
from torch.distributions import MultivariateNormal

mu = torch.tensor([0.0, 0.0], dtype=torch.float)
C = torch.tensor([[5.0, 0.0], [0.0, 5.0]], dtype=torch.float)

mvn_dist = MultivariateNormal(mu, C)

X = torch.tensor([0.0, 0.0], dtype=torch.float)

def formula(X, mu, C):
    K = (1 / (2 * math.pi * torch.sqrt(C.det())))
    X_minus_mu = (X - mu).reshape(-1, 1)
    E1 = torch.matmul(X_minus_mu.T, C.inverse())
    E = torch.exp(-1 / 2. * torch.matmul(E1, X_minus_mu))
    return torch.log(K * E)

log_prob = mvn_dist.log_prob(X)

formula_log_prob = formula(X, mu, C)

assert torch.isclose(log_prob, formula_log_prob, atol=1e-4)

# Sampling

num_samples = 100000

samples = mvn_dist.sample([num_samples])

# Mean
    
sample_mean = samples.mean()

dist_mean = mvn_dist.mean

assert torch.allclose(sample_mean, dist_mean, atol=1e-1)

# Variance

sample_var = mvn_dist.sample([num_samples]).var()

dist_var = mvn_dist.variance

assert torch.allclose(sample_var, dist_var, atol=1e-1)

Here we allow the user to set different values for the means and covariance matrix of a 2D Normal distribution and visualise the resulting distribution. 

Specifically, notice that changing the mean does not change the shape of the distribution. It just varies where the distribution peaks. Changing $\mu_{0}$ shifts the center along the X axis. Similarly changing $\mu_{1}$ shifts the center along the Y-axis

While providing values for the covariance matrix, we should ensure that the matrix is not singular.

In [5]:
from mpl_toolkits.mplot3d import Axes3D # <--- This is important for 3d plotting 
from matplotlib import cm

fig_1, ax_1 = plt.subplots(nrows=1, ncols=1)
ax_1.set_title("2D normal distribution")
ax_1 = fig_1.gca(projection='3d')


@interact
def plot_2d_normal(mu0=(-5, 5, 0.25), mu1=(-5, 5, 0.25),
                   sigma_00=widgets.FloatText(1.0), 
                   sigma_01=widgets.FloatText(0.0), 
                   sigma_11 = widgets.FloatText(1.0)):
    
    def _reset_plot(ax):
        ax.clear()
        ax.set_ylabel("Y")
        ax.set_xlabel("X")
        ax.set_zlabel("P(X,Y)")
        
    X = np.linspace(-10, 10, 1000)
    Y = np.linspace(-10, 10, 1000)
    X, Y = np.meshgrid(X, Y)
    XY = np.stack((X, Y), axis=2)
    mu = np.array([mu0, mu1])
    sigma_10 = sigma_01 # Covariance matrix is symmetrical

    C = np.array([[sigma_00, sigma_01], [sigma_10, sigma_11]])
    try:
        mvn_dist = MultivariateNormal(torch.from_numpy(mu), torch.from_numpy(C))
        Z = mvn_dist.log_prob(torch.from_numpy(XY)).exp()
        # Plot the surface.
        _reset_plot(ax_1)
        ax_1.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    except RuntimeError:
        print("Error! :Covariance matrix cannot be singular!")
        ax_1.clear()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

/Users/ananya.h.a/Documents/venvs/dlbook/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Requested projection is different from current axis projection, creating new axis with requested projection.
  


interactive(children=(FloatSlider(value=0.0, description='mu0', max=5.0, min=-5.0, step=0.25), FloatSlider(val…

### Contour plots obtained from 2-D normal distributions

If $\sum$ is a diagonal matrix with equal diagonal elements, the bell is symmetric in all directions,
its base is circular

If $\sum$ is a diagonal matrix with unequal diagonal elements, the base of the bell is elliptical.
The axes of the ellipse are aligned with coordinate axes.

For general $\sum$ matrix the base of the bell is elliptical. The axes of the ellipse are not necessarily
aligned with coordinate axes.

In [6]:
fig_2, ax_2 = plt.subplots(nrows=1, ncols=1)


@interact
def plot_2d_normal_contour(mu0=(-5, 5, 0.25), mu1=(-5, 5, 0.25), 
                           sigma_00=widgets.FloatText(1.0), 
                           sigma_01=widgets.FloatText(0.0), 
                           sigma_11 = widgets.FloatText(1.0)):
    def _reset_plot(ax):
        ax.clear()
        ax_2.set_title("2D normal distribution contour plot")
        
    X = np.linspace(-10, 10, 1000)
    Y = np.linspace(-10, 10, 1000)
    X, Y = np.meshgrid(X, Y)
    XY = np.stack((X, Y), axis=2)
    mu = np.array([mu0, mu1])
    sigma_10 = sigma_01 # Covariance matrix is symmetrical

    C = np.array([[sigma_00, sigma_01], [sigma_10, sigma_11]])
    try:
        mvn_dist = MultivariateNormal(torch.from_numpy(mu), torch.from_numpy(C))
        Z = mvn_dist.log_prob(torch.from_numpy(XY)).exp()
        _reset_plot(ax_2)
        ax_2.contour(Z)
    except RuntimeError:
        print("Error!: The covariance matrix must not be singular")
        ax_2.clear()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=0.0, description='mu0', max=5.0, min=-5.0, step=0.25), FloatSlider(val…